## Importing Modules

In [1]:
import os
import pathlib
import numpy as np
import pandas as pd
import nltk

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline, GPT2Config
from tqdm.auto import tqdm
import random
import datetime
import time
import statistics
from nltk.translate.bleu_score import sentence_bleu
from transformers import TrainingArguments, Trainer
from datasets import load_dataset

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available.")

d:\Python\LLM_Environment\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



GPU is available!


In [2]:
MAIN_PATH = str(pathlib.Path().resolve())
DATASET_PATH = MAIN_PATH + '\\datasets'
MODEL_PATH = MAIN_PATH + '\\models'

In [3]:
models = os.listdir(MODEL_PATH)
models

['bert-base-cased',
 'bert-base-multilingual-cased',
 'bert-base-uncased',
 'bert-large-cased',
 'bert-large-uncased',
 'flan-t5-base',
 'flan-t5-large',
 'flan-t5-small',
 'gpt2',
 'gpt2-large',
 'gpt2-medium',
 'tuned_text_gen']

In [4]:
model_path = MODEL_PATH + '\\' + models[8]
model_path

'D:\\Python\\LLM_Environment\\models\\gpt2'

In [5]:
model = GPT2LMHeadModel.from_pretrained(model_path)

In [6]:
torch.cuda.empty_cache()

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [8]:
# Place the model in evaluation mode
model.eval()

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Your prompt for the recipe
recipe_prompt = "Give me a recipe instructions to make a cake"

# Tokenize the input, this encodes the prompt to a format that the model can understand
inputs = tokenizer.encode(recipe_prompt, return_tensors='pt').to(device)

# Generate the output using the model
outputs = model.generate(inputs, max_length=512, num_return_sequences=1)

# Decode the generated output back to a readable string
recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(recipe)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
d:\Python\LLM_Environment\myenv\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Give me a recipe instructions to make a cake.

I've been making this for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I've been really enjoying it. I've been making it for a while now and I

In [9]:
# Your initial instructions string
instructions_string = f"""give me a recipe to make a chocolate cake"""
comment ="To make a basic chocolate cake, mix 1 cup sugar, ¾ cup flour, ½ cup cocoa powder, ¾ tsp baking powder, ¾ tsp baking soda, a pinch of salt, 1 egg, ½ cup milk, ¼ cup vegetable oil, and 1 tsp vanilla extract. Bake at 350°F (175°C) for 30-35 minutes. Enjoy your delicious dessert!"

# Prompt template
prompt_template = lambda instructions_string, comment: f'''[INST] {instructions_string} \nInstructions: {comment} \n[/INST]'''

# Create the prompt
prompt = prompt_template(instructions_string, comment)
print(prompt)

[INST] give me a recipe to make a chocolate cake 
Instructions: To make a basic chocolate cake, mix 1 cup sugar, ¾ cup flour, ½ cup cocoa powder, ¾ tsp baking powder, ¾ tsp baking soda, a pinch of salt, 1 egg, ½ cup milk, ¼ cup vegetable oil, and 1 tsp vanilla extract. Bake at 350°F (175°C) for 30-35 minutes. Enjoy your delicious dessert! 
[/INST]


In [10]:
# Tokenize the input text using the GPT-2 tokenizer
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the output using the GPT-2 model
outputs = model.generate(input_ids=inputs["input_ids"], max_length=512) 

# Decode the generated output back to a readable string, removing any special tokens
recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(recipe)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[INST] give me a recipe to make a chocolate cake 
Instructions: To make a basic chocolate cake, mix 1 cup sugar, ¾ cup flour, ½ cup cocoa powder, ¾ tsp baking powder, ¾ tsp baking soda, a pinch of salt, 1 egg, ½ cup milk, ¼ cup vegetable oil, and 1 tsp vanilla extract. Bake at 350°F (175°C) for 30-35 minutes. Enjoy your delicious dessert! 
[/INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]
[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INST]

[INS

In [11]:
# Set the pad_token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Ensure the model and tokenizer are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Your prompt for the recipe
prompt = "Give me a recipe to make a chocolate cake"

# Tokenize the input text using the GPT-2 tokenizer
inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", max_length=512, truncation=True).to(device)

# Generate the output using the GPT-2 model
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=50  # Specify how many new tokens to generate beyond the input
)

# Decode the generated output back to a readable string, removing any special tokens
recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(recipe)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Give me a recipe to make a chocolate cake

I've been making chocolate cake for a while now, and I've been making it for a long time. I've been making it for a long time, and I've been making it for a long time. I've been making it


In [12]:
config = GPT2Config.from_pretrained(model_path)

if hasattr(model, 'gradient_checkpointing_enable'):
    model.gradient_checkpointing_enable()

In [13]:
# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

# Total number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable parameters:", trainable_params)

# Percentage of trainable parameters
trainable_percentage = (trainable_params / total_params) * 100
print("Trainable percentage: {:.2f}%".format(trainable_percentage))

Total parameters: 124439808
Trainable parameters: 124439808
Trainable percentage: 100.00%


## Import Dataset

In [14]:
# load dataset
data = load_dataset("darkraipro/recipe-instructions")

In [15]:
def convert_to_tensors(batch):
    # Convert list of integers in the batch to a tensor, individually for each example
    batch['input_ids'] = [torch.tensor(e, dtype=torch.long) for e in batch['input_ids']]
    batch['attention_mask'] = [torch.tensor(e, dtype=torch.long) for e in batch['attention_mask']]
    return batch

# Note that batched=True applies the function to each batch of examples
dataset = data.map(convert_to_tensors, batched=True)

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 339583
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 17934
    })
})

In [17]:
# Initialize data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [18]:
save_path = './model'
# Define your training arguments
training_args = TrainingArguments(
    output_dir=save_path,
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Small batch size
    gradient_accumulation_steps=8,  # Accumulate gradients to effectively have batch size of 8
    fp16=True,  # Enable FP16 training
    gradient_checkpointing=True
)

In [19]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"].select(range(10000)),
)

In [20]:
# Train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

  0%|          | 0/468 [00:00<?, ?it/s]d:\Python\LLM_Environment\myenv\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
100%|██████████| 468/468 [21:10<00:00,  2.72s/it]

{'train_runtime': 1270.9911, 'train_samples_per_second': 23.604, 'train_steps_per_second': 0.368, 'train_loss': 2.4751427805321846, 'epoch': 3.0}


TrainOutput(global_step=468, training_loss=2.4751427805321846, metrics={'train_runtime': 1270.9911, 'train_samples_per_second': 23.604, 'train_steps_per_second': 0.368, 'total_flos': 7826218942464000.0, 'train_loss': 2.4751427805321846, 'epoch': 2.9952})

In [21]:
# renable warnings
model.config.use_cache = True

In [22]:
model = GPT2LMHeadModel.from_pretrained(save_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")


# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_path, use_fast=True)

Some parameters are on the meta device because they were offloaded to the cpu.


In [24]:
intstructions_string = f"""Can you give me the recipe to make a chocolate cake?"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

comment = "To make a basic chocolate cake, mix 1 cup sugar, ¾ cup flour, ½ cup cocoa powder, ¾ tsp baking powder, ¾ tsp baking soda, a pinch of salt, 1 egg, ½ cup milk, ¼ cup vegetable oil, and 1 tsp vanilla extract. Bake at 350°F (175°C) for 30-35 minutes. Enjoy your delicious dessert!"

prompt = prompt_template(comment)
print(prompt)

[INST] Can you give me the recipe to make a chocolate cake? 
To make a basic chocolate cake, mix 1 cup sugar, ¾ cup flour, ½ cup cocoa powder, ¾ tsp baking powder, ¾ tsp baking soda, a pinch of salt, 1 egg, ½ cup milk, ¼ cup vegetable oil, and 1 tsp vanilla extract. Bake at 350°F (175°C) for 30-35 minutes. Enjoy your delicious dessert! 
[/INST]


In [25]:
model.eval()

# inputs = tokenizer(prompt, return_tensors="pt")
# outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
# print(tokenizer.batch_decode(outputs)[0])


# Tokenize the input text using the GPT-2 tokenizer
inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", max_length=512, truncation=True).to(device)

# Generate the output using the GPT-2 model
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=50  # Specify how many new tokens to generate beyond the input
)

# Decode the generated output back to a readable string, removing any special tokens
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[INST] Can you give me the recipe to make a chocolate cake? 
To make a basic chocolate cake, mix 1 cup sugar, ¾ cup flour, ½ cup cocoa powder, ¾ tsp baking powder, ¾ tsp baking soda, a pinch of salt, 1 egg, ½ cup milk, ¼ cup vegetable oil, and 1 tsp vanilla extract. Bake at 350°F (175°C) for 30-35 minutes. Enjoy your delicious dessert! 
[/INST]:                                                 


In [26]:
comment = "Can you give me the recipe to make an benedict eggs?"
prompt = prompt_template(comment)

model.eval()
# inputs = tokenizer(prompt, return_tensors="pt")

# outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
# print(tokenizer.batch_decode(outputs)[0])


inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", max_length=512, truncation=True).to(device)
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=50  # Specify how many new tokens to generate beyond the input
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[INST] Can you give me the recipe to make a chocolate cake? 
Can you give me the recipe to make an benedict eggs? 
[/INST]: 
 and the first-1, the first-1, the first-1, the first-1, the first-in, the first-in, the first-in
